In [1]:
"""
This is a skeleton of Assignment 2 for DS5110 Spring 2023.
NOTE you will need to change the .ipynb file name by following the naming convention.
Code should be commented well. 
Feel free to import any missing packages. 
"""
import dask
from dask.distributed import Client
import dask.dataframe as dd
import time
import json
import numpy as np

In [2]:
# TODO: fill out scheduler IP and port here when creating a Dask client object
client = Client('172.31.69.136:8786')
client = client.restart()
print(client)

<Client: 'tcp://172.31.69.136:8786' processes=8 threads=32, memory=61.31 GiB>


In [3]:
%%time

# Load the .CSV files into the Dask dataframes
questions = dd.read_csv('/home/ubuntu/questions.csv', sample=2**30, assume_missing=True)
question_tags = dd.read_csv('/home/ubuntu/question_tags.csv', sample=2**30, assume_missing=True)

CPU times: user 694 ms, sys: 2.18 s, total: 2.87 s
Wall time: 2.87 s


In [5]:
questions

,Id,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount
npartitions=14,,,,,,,
,float64,object,object,object,float64,float64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [38]:
%%time
# Task 1

# TODO: Start your implementation below this line
questions_missing = questions.isna().mean().compute() * 100
question_tags_missing = question_tags.isna().mean().compute() * 100

print('Questions Missing')
print(questions_missing)

print('Questions Tags Missing')
print(question_tags_missing)

Questions Missing
Id               0.000000
CreationDate     0.000000
ClosedDate      89.973578
DeletionDate    76.962907
Score            0.000000
OwnerUserId     23.920885
AnswerCount     10.114036
dtype: float64
Questions Tags Missing
Id     0.000000
Tag    0.027501
dtype: float64
CPU times: user 85.2 ms, sys: 13.4 ms, total: 98.6 ms
Wall time: 24.5 s


In [39]:
%%time
# Task 2

# TODO: Start your implementation below this line

# calculate various summary statistics for column 'A'
mean_score = questions['Score'].mean().compute()
std_score = questions['Score'].std().compute()
median_score = questions['Score'].median_approximate().compute()
min_score = questions['Score'].min().compute()
max_score = questions['Score'].max().compute()

# print the results
print('Mean Score:', mean_score)
print('Standard deviation of Score:', std_score)
print('Median Score:', median_score)
print('Min Score:', min_score)
print('Max Score:', max_score)


Mean Score: 1.2041671665555287
Standard deviation of Score: 15.64573552737141
Median Score: 1.0
Min Score: -154.0
Max Score: 16902.0
CPU times: user 113 ms, sys: 9.9 ms, total: 123 ms
Wall time: 19.7 s


In [40]:
%%time
# Task 3

# TODO: Start your implementation below this line
tag_counts = question_tags.groupby('Tag').count()
sorted_tags = tag_counts.compute().sort_values(by='Id', ascending=False)
top_5_tags = sorted_tags.head(5)
print(top_5_tags)


                 Id
Tag                
javascript  1649631
java        1563820
php         1365600
c#          1321027
android     1288338
CPU times: user 43.2 ms, sys: 11 ms, total: 54.2 ms
Wall time: 11.6 s


In [41]:
%%time
# Task 4

# TODO: Start your implementation below this line

# join the dataframes and create a boolean mask
joined = questions.merge(question_tags, left_on='Id', right_on='Id', how='left')
mask = joined['Tag'].isna()

# tag the questions
tagged = questions.assign(tagged=mask.astype(int))
tag_counts = tagged.groupby('tagged')['Id'].count()

# print the result
print(tag_counts.compute())

tagged
0    17198444
1        5380
Name: Id, dtype: int64
CPU times: user 83 ms, sys: 1.65 ms, total: 84.6 ms
Wall time: 1min


In [42]:
%%time
# Task 5

# TODO: Start your implementation below this line

# Group the questions by OwnerUserId and calculate the aggregated values
owner_user_df = questions.groupby('OwnerUserId').agg({
    'Score': 'mean',
    'Id': 'count',
    'AnswerCount': 'sum'
})

# Rename the columns to match the required output
owner_user_df = owner_user_df.rename(columns={
    'Score': 'AverageScore',
    'Id': 'NumQuestions',
    'AnswerCount': 'NumAnswers'
})

# Reset the index to turn the OwnerUserId column into a regular column
owner_user_df = owner_user_df.reset_index()

# Display the resulting owner user table
print(owner_user_df.head())


   OwnerUserId  AverageScore  NumQuestions  NumAnswers
0          8.0    458.000000             1        13.0
1          9.0     37.052632            19        66.0
2          1.0    298.400000            15       344.0
3         11.0    290.000000             1         8.0
4          2.0     31.428571             7        28.0
CPU times: user 50.1 ms, sys: 1.42 ms, total: 51.5 ms
Wall time: 11.1 s


In [37]:
%%time
# Task 6

# TODO: Start your implementation below this line

# Merge the two tables using Id as the index
merged_df = questions.merge(question_tags, on='Id')

# Group the merged table by Tag and calculate the aggregated values
tag_df = merged_df.groupby('Tag').agg({
    'Score': 'mean',
    'AnswerCount': 'sum',
    'Id': 'count',
    'OwnerUserId': 'count'
})

# Rename the columns to match the required output
tag_df = tag_df.rename(columns={
    'Score': 'AverageScore',
    'AnswerCount': 'NumAnswers',
    'Id': 'NumQuestions',
    'OwnerUserId': 'NumOwners'
})

# Reset the index to turn the Tag column into a regular column
tag_df = tag_df.reset_index()

# Display the resulting table
print(tag_df.head())


            Tag  AverageScore  NumAnswers  NumQuestions  NumOwners
0           php      0.375525   1855546.0       1365600    1017270
1       plugins      0.903577     25199.0         27452      17995
2  architecture      2.018781     25229.0         16293      11549
3          hook      1.144506      4854.0          4851       3581
4          linq      1.824996    123939.0         69770      61034
CPU times: user 89.3 ms, sys: 12 ms, total: 101 ms
Wall time: 59.9 s
